In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.transforms as T
import numpy as np
import h5py
import logging
import os
import sys
import datetime

In [3]:
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, train):
        super(EEGDataset).__init__()
        assert x.shape[0] == y.size
        self.x = x
        #temp_y = np.zeros((y.size, 2))
        #for i in range(y.size):
        #    temp_y[i, y[i]] = 1
        #self.y = temp_y
        self.y = [y[i][0] for i in range(y.size)]
        self.train = train

    def __getitem__(self,key):
        return (self.x[key], self.y[key])

    def __len__(self):
        return len(self.y)

In [4]:

# Load EEG data
transform = T.Compose([
    T.ToTensor()
])
f = h5py.File('child_mind_x_train_v2.mat', 'r')
x_train = f['X_train']
x_train = np.reshape(x_train,(-1,1,24,256))
print('X_train shape: ' + str(x_train.shape))
f = h5py.File('child_mind_y_train_v2.mat', 'r')
y_train = f['Y_train']
print('Y_train shape: ' + str(y_train.shape))
train_data = EEGDataset(x_train, y_train, True)


f = h5py.File('child_mind_x_val_v2.mat', 'r')
x_val = f['X_val']
x_val = np.reshape(x_val,(-1,1,24,256))
print('X_val shape: ' + str(x_val.shape))
f = h5py.File('child_mind_y_val_v2.mat', 'r')
y_val = f['Y_val']
print('Y_val shape: ' + str(y_val.shape))
val_data = EEGDataset(x_val, y_val, True)

f = h5py.File('child_mind_x_test_v2.mat', 'r')
x_test = f['X_test']
x_test = np.reshape(x_test,(-1,1,24,256))
print('X_test shape: ' + str(x_test.shape))
f = h5py.File('child_mind_y_test_v2.mat', 'r')
y_test = f['Y_test']
print('Y_test shape: ' + str(y_test.shape))
test_data = EEGDataset(x_test, y_test, False)
loader_test = DataLoader(test_data, batch_size=70)

X_train shape: (71300, 1, 24, 256)
Y_train shape: (71300, 1)
X_val shape: (39868, 1, 24, 256)
Y_val shape: (39868, 1)
X_test shape: (16006, 1, 24, 256)
Y_test shape: (16006, 1)


In [5]:
print(np.histogram(y_train))

(array([35626,     0,     0,     0,     0,     0,     0,     0,     0,
       35674]), array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))


In [5]:
# Test with MNIST
'''
import torchvision.datasets as dset
NUM_TRAIN = 40000
transform = T.Compose([
                T.ToTensor(),
                T.CenterCrop(24),
                T.Pad((116,0))
            ])
mnist_train = dset.MNIST('./mnist', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(mnist_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

mnist_val = dset.MNIST('./mnist', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(mnist_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 60000)))

mnist_test = dset.MNIST('./mnist', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(mnist_test, batch_size=64)
'''

"\nimport torchvision.datasets as dset\nNUM_TRAIN = 40000\ntransform = T.Compose([\n                T.ToTensor(),\n                T.CenterCrop(24),\n                T.Pad((116,0))\n            ])\nmnist_train = dset.MNIST('./mnist', train=True, download=True,\n                             transform=transform)\nloader_train = DataLoader(mnist_train, batch_size=64, \n                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))\n\nmnist_val = dset.MNIST('./mnist', train=True, download=True,\n                           transform=transform)\nloader_val = DataLoader(mnist_val, batch_size=64, \n                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 60000)))\n\nmnist_test = dset.MNIST('./mnist', train=False, download=True, \n                            transform=transform)\nloader_test = DataLoader(mnist_test, batch_size=64)\n"

In [8]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [9]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [18]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                check_accuracy(loader_val, model)
                print()
                
        if e % 10 == 0:
            print('Save model at epoch %d' % (e))
            torch.save(model.state_dict(), 'logs/model_saved-bs32-temp')

In [ ]:
log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join('logs', f'log-fine-tune-0.000363-32.txt'))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

for i in range(25):
    r = -4 * (1-0.25*np.random.rand())
    lr = 10**r
    #lr = 0.000363
    batch_size = 2**np.random.randint(2,7)
    #batch_size = 32
    logging.info('Learning rate: %f, batch_size: %d' % (lr, batch_size))
    loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(val_data, batch_size=batch_size)
    model = nn.Sequential(
        nn.Conv2d(1,100,3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(100,100,3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(100,300,(2,3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(300,300,(1,7)),
        nn.ReLU(),
        nn.MaxPool2d((1,2), stride=1),
        nn.Dropout(0.25),
        nn.Conv2d(300,100,(1,3)),
        nn.Conv2d(100,100,(1,3)),
        nn.Flatten(),
        nn.Linear(1900,6144),
        nn.Linear(6144,2),
    )

    # pred = model(next(iter(loader_train))[0])
    #%%
    # print(pred.shape)
    #%%
    optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
    train(model, optimizer, epochs=25)
    check_accuracy(loader_test, model)
    torch.save(model.state_dict(), f'logs/model_saved-lr{lr}-bs{batch_size}-{datetime.datetime.today()}')

03/15 10:38:38 PM Learning rate: 0.000363, batch_size: 32
Iteration 0, loss = 1.4855
Checking accuracy on validation set
Got 19910 / 39868 correct (49.94)

Iteration 100, loss = 0.5894
Checking accuracy on validation set
Got 20449 / 39868 correct (51.29)

Iteration 200, loss = 0.7102
Checking accuracy on validation set
Got 20024 / 39868 correct (50.23)

Iteration 300, loss = 0.6788
Checking accuracy on validation set
Got 21538 / 39868 correct (54.02)

Iteration 400, loss = 0.6577
Checking accuracy on validation set
Got 20451 / 39868 correct (51.30)

Iteration 500, loss = 0.6818
Checking accuracy on validation set
Got 19988 / 39868 correct (50.14)

Iteration 600, loss = 0.6616
Checking accuracy on validation set
Got 20574 / 39868 correct (51.61)

Iteration 700, loss = 0.6864
Checking accuracy on validation set
Got 20010 / 39868 correct (50.19)

Iteration 800, loss = 0.6850
Checking accuracy on validation set
Got 20015 / 39868 correct (50.20)

Iteration 900, loss = 0.6655
Checking accura

Got 25494 / 39868 correct (63.95)

Iteration 1300, loss = 0.9916
Checking accuracy on validation set
Got 25686 / 39868 correct (64.43)

Iteration 1400, loss = 0.4929
Checking accuracy on validation set
Got 25596 / 39868 correct (64.20)

Iteration 1500, loss = 0.5201
Checking accuracy on validation set
Got 26064 / 39868 correct (65.38)

Iteration 1600, loss = 0.7018
Checking accuracy on validation set
Got 24938 / 39868 correct (62.55)

Iteration 1700, loss = 0.5997
Checking accuracy on validation set
Got 25108 / 39868 correct (62.98)

Iteration 1800, loss = 0.6806
Checking accuracy on validation set
Got 25523 / 39868 correct (64.02)

Iteration 1900, loss = 0.5341
Checking accuracy on validation set
Got 25893 / 39868 correct (64.95)

Iteration 2000, loss = 0.8539
Checking accuracy on validation set
Got 25967 / 39868 correct (65.13)

Iteration 2100, loss = 0.5384
Checking accuracy on validation set
Got 25892 / 39868 correct (64.94)

Iteration 2200, loss = 0.6412
Checking accuracy on valid

## Fine tune model
lr = 0.000363
batch_size = 32
* Continue training after 25 epochs

In [16]:
model = nn.Sequential(
        nn.Conv2d(1,100,3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(100,100,3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(100,300,(2,3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(300,300,(1,7)),
        nn.ReLU(),
        nn.MaxPool2d((1,2), stride=1),
        nn.Dropout(0.25),
        nn.Conv2d(300,100,(1,3)),
        nn.Conv2d(100,100,(1,3)),
        nn.Flatten(),
        nn.Linear(1900,6144),
        nn.Linear(6144,2),
    )
model = model.to(device=device)
model.load_state_dict(torch.load('logs/model_saved-lr0.000363-bs32-2021-03-16 00:06:28.364050'))
model.eval()


Sequential(
  (0): Conv2d(1, 100, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Dropout(p=0.25, inplace=False)
  (4): Conv2d(100, 100, kernel_size=(3, 3), stride=(1, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.25, inplace=False)
  (8): Conv2d(100, 300, kernel_size=(2, 3), stride=(1, 1))
  (9): ReLU()
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Dropout(p=0.25, inplace=False)
  (12): Conv2d(300, 300, kernel_size=(1, 7), stride=(1, 1))
  (13): ReLU()
  (14): MaxPool2d(kernel_size=(1, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
  (15): Dropout(p=0.25, inplace=False)
  (16): Conv2d(300, 100, kernel_size=(1, 3), stride=(1, 1))
  (17): Conv2d(100, 100, kernel_size=(1, 3), stride=(1, 1))
  (18): Flatten()
  (19): Linear(in_features=1900, out_features=6144, bias=True)
  (20)

In [17]:
# confirm state of model
check_accuracy(loader_test, model)

Checking accuracy on test set
Got 11531 / 16006 correct (72.04)


In [ ]:
lr = 0.000363
batch_size = 32
loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_data, batch_size=batch_size)
optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
train(model, optimizer, epochs=100)
check_accuracy(loader_test, model)
torch.save(model.state_dict(), f'logs/model_saved-lr{lr}-bs{batch_size}-{datetime.datetime.today()}')

Epoch 0, Iteration 0, loss = 0.3884
Checking accuracy on validation set
Got 20822 / 39868 correct (52.23)

Epoch 0, Iteration 100, loss = 0.4649
Checking accuracy on validation set
Got 28732 / 39868 correct (72.07)

Epoch 0, Iteration 200, loss = 0.5119
Checking accuracy on validation set
Got 28476 / 39868 correct (71.43)

Epoch 0, Iteration 300, loss = 0.4005
Checking accuracy on validation set
Got 28877 / 39868 correct (72.43)

Epoch 0, Iteration 400, loss = 0.3551
Checking accuracy on validation set
Got 28661 / 39868 correct (71.89)

Epoch 0, Iteration 500, loss = 0.6896
Checking accuracy on validation set
Got 28766 / 39868 correct (72.15)

Epoch 0, Iteration 600, loss = 0.4052
Checking accuracy on validation set
Got 28530 / 39868 correct (71.56)

Epoch 0, Iteration 700, loss = 0.4052
Checking accuracy on validation set
Got 28912 / 39868 correct (72.52)

Epoch 0, Iteration 800, loss = 0.3644
Checking accuracy on validation set
Got 28646 / 39868 correct (71.85)

Epoch 0, Iteration 90

Got 28939 / 39868 correct (72.59)

Epoch 3, Iteration 600, loss = 0.3363
Checking accuracy on validation set
Got 29023 / 39868 correct (72.80)

Epoch 3, Iteration 700, loss = 0.4102
Checking accuracy on validation set
Got 28638 / 39868 correct (71.83)

Epoch 3, Iteration 800, loss = 0.3799
Checking accuracy on validation set
Got 28969 / 39868 correct (72.66)

Epoch 3, Iteration 900, loss = 0.4755
Checking accuracy on validation set
Got 29037 / 39868 correct (72.83)

Epoch 3, Iteration 1000, loss = 0.3905
Checking accuracy on validation set
Got 29038 / 39868 correct (72.84)

Epoch 3, Iteration 1100, loss = 0.3810
Checking accuracy on validation set
Got 28946 / 39868 correct (72.60)

Epoch 3, Iteration 1200, loss = 0.3322
Checking accuracy on validation set
Got 28859 / 39868 correct (72.39)

Epoch 3, Iteration 1300, loss = 0.5189
Checking accuracy on validation set
Got 28899 / 39868 correct (72.49)

Epoch 3, Iteration 1400, loss = 0.4641
Checking accuracy on validation set
Got 28359 / 39

Got 28852 / 39868 correct (72.37)

Epoch 6, Iteration 1200, loss = 0.4053
Checking accuracy on validation set
Got 29139 / 39868 correct (73.09)

Epoch 6, Iteration 1300, loss = 0.4428
Checking accuracy on validation set
Got 29205 / 39868 correct (73.25)

Epoch 6, Iteration 1400, loss = 0.4572
Checking accuracy on validation set
Got 28816 / 39868 correct (72.28)

Epoch 6, Iteration 1500, loss = 0.3604
Checking accuracy on validation set
Got 28993 / 39868 correct (72.72)

Epoch 6, Iteration 1600, loss = 0.3555
Checking accuracy on validation set
Got 28876 / 39868 correct (72.43)

Epoch 6, Iteration 1700, loss = 0.4131
Checking accuracy on validation set
Got 28915 / 39868 correct (72.53)

Epoch 6, Iteration 1800, loss = 0.3190
Checking accuracy on validation set
Got 28976 / 39868 correct (72.68)

Epoch 6, Iteration 1900, loss = 0.4449
Checking accuracy on validation set
Got 29036 / 39868 correct (72.83)

Epoch 6, Iteration 2000, loss = 0.5462
Checking accuracy on validation set
Got 28612 

Got 29188 / 39868 correct (73.21)

Epoch 9, Iteration 1800, loss = 0.3090
Checking accuracy on validation set
Got 29306 / 39868 correct (73.51)

Epoch 9, Iteration 1900, loss = 0.3719
Checking accuracy on validation set
Got 29246 / 39868 correct (73.36)

Epoch 9, Iteration 2000, loss = 0.2279
Checking accuracy on validation set
Got 29285 / 39868 correct (73.45)

Epoch 9, Iteration 2100, loss = 0.4049
Checking accuracy on validation set
Got 29171 / 39868 correct (73.17)

Epoch 9, Iteration 2200, loss = 0.3024
Checking accuracy on validation set
Got 28950 / 39868 correct (72.61)

Epoch 10, Iteration 0, loss = 0.2834
Checking accuracy on validation set
Got 28851 / 39868 correct (72.37)

Epoch 10, Iteration 100, loss = 0.4506
Checking accuracy on validation set
Got 28785 / 39868 correct (72.20)

Epoch 10, Iteration 200, loss = 0.6829
Checking accuracy on validation set
Got 28580 / 39868 correct (71.69)

Epoch 10, Iteration 300, loss = 0.3840
Checking accuracy on validation set
Got 29035 / 

Got 29260 / 39868 correct (73.39)

Epoch 13, Iteration 0, loss = 0.4047
Checking accuracy on validation set
Got 29205 / 39868 correct (73.25)

Epoch 13, Iteration 100, loss = 0.2343
Checking accuracy on validation set
Got 29334 / 39868 correct (73.58)

Epoch 13, Iteration 200, loss = 0.2259
Checking accuracy on validation set
Got 29013 / 39868 correct (72.77)

Epoch 13, Iteration 300, loss = 0.5724
Checking accuracy on validation set
Got 28895 / 39868 correct (72.48)

Epoch 13, Iteration 400, loss = 0.3664
Checking accuracy on validation set
Got 29193 / 39868 correct (73.22)

Epoch 13, Iteration 500, loss = 0.5072
Checking accuracy on validation set
Got 29384 / 39868 correct (73.70)

Epoch 13, Iteration 600, loss = 0.3053
Checking accuracy on validation set
Got 29258 / 39868 correct (73.39)

Epoch 13, Iteration 700, loss = 0.3496
Checking accuracy on validation set
Got 29411 / 39868 correct (73.77)

Epoch 13, Iteration 800, loss = 0.4684
Checking accuracy on validation set
Got 29280 / 

...

Epoch 75, Iteration 1300, loss = 0.2503
Checking accuracy on validation set
Got 28059 / 39868 correct (70.38)

Epoch 75, Iteration 1400, loss = 0.3168
Checking accuracy on validation set
Got 29138 / 39868 correct (73.09)

Epoch 75, Iteration 1500, loss = 0.3140
Checking accuracy on validation set
Got 28853 / 39868 correct (72.37)

Epoch 75, Iteration 1600, loss = 0.3983
Checking accuracy on validation set
Got 29197 / 39868 correct (73.23)

Epoch 75, Iteration 1700, loss = 0.2254
Checking accuracy on validation set
Got 28822 / 39868 correct (72.29)

Epoch 75, Iteration 1800, loss = 0.3892
Checking accuracy on validation set
Got 28982 / 39868 correct (72.69)

Epoch 75, Iteration 1900, loss = 0.1611
Checking accuracy on validation set
Got 29205 / 39868 correct (73.25)

Epoch 75, Iteration 2000, loss = 0.2946
Checking accuracy on validation set
Got 29192 / 39868 correct (73.22)

Epoch 75, Iteration 2100, loss = 0.2569
Checking accuracy on validation set
Got 29002 / 39868 correct (72.75)

E

Got 29051 / 39868 correct (72.87)

Epoch 78, Iteration 1900, loss = 0.2742
Checking accuracy on validation set
Got 28690 / 39868 correct (71.96)

Epoch 78, Iteration 2000, loss = 0.3771
Checking accuracy on validation set
Got 28681 / 39868 correct (71.94)

Epoch 78, Iteration 2100, loss = 0.3849
Checking accuracy on validation set
Got 29113 / 39868 correct (73.02)

Epoch 78, Iteration 2200, loss = 0.2688
Checking accuracy on validation set
Got 29163 / 39868 correct (73.15)

Epoch 79, Iteration 0, loss = 0.2027
Checking accuracy on validation set
Got 28821 / 39868 correct (72.29)

Epoch 79, Iteration 100, loss = 0.1927
Checking accuracy on validation set
Got 29144 / 39868 correct (73.10)

Epoch 79, Iteration 200, loss = 0.3269
Checking accuracy on validation set
Got 28938 / 39868 correct (72.58)

Epoch 79, Iteration 300, loss = 0.3342
Checking accuracy on validation set
Got 29045 / 39868 correct (72.85)

Epoch 79, Iteration 400, loss = 0.3379
Checking accuracy on validation set
Got 2890

Got 28905 / 39868 correct (72.50)

Epoch 82, Iteration 100, loss = 0.3104
Checking accuracy on validation set
Got 29002 / 39868 correct (72.75)

Epoch 82, Iteration 200, loss = 0.1396
Checking accuracy on validation set
Got 29053 / 39868 correct (72.87)

Epoch 82, Iteration 300, loss = 0.1926
Checking accuracy on validation set
Got 29020 / 39868 correct (72.79)

Epoch 82, Iteration 400, loss = 0.2027
Checking accuracy on validation set
Got 28558 / 39868 correct (71.63)

Epoch 82, Iteration 500, loss = 0.3426
Checking accuracy on validation set
Got 28337 / 39868 correct (71.08)

Epoch 82, Iteration 600, loss = 0.4392
Checking accuracy on validation set
Got 28931 / 39868 correct (72.57)

Epoch 82, Iteration 700, loss = 0.1700
Checking accuracy on validation set
Got 28935 / 39868 correct (72.58)

Epoch 82, Iteration 800, loss = 0.1257
Checking accuracy on validation set
Got 28760 / 39868 correct (72.14)

Epoch 82, Iteration 900, loss = 0.1891
Checking accuracy on validation set
Got 28979 

Epoch 85, Iteration 600, loss = 0.5909
Checking accuracy on validation set
Got 29021 / 39868 correct (72.79)

Epoch 85, Iteration 700, loss = 0.3043
Checking accuracy on validation set
Got 29025 / 39868 correct (72.80)

Epoch 85, Iteration 800, loss = 0.3836
Checking accuracy on validation set
Got 28876 / 39868 correct (72.43)

Epoch 85, Iteration 900, loss = 0.2207
Checking accuracy on validation set
Got 29185 / 39868 correct (73.20)

Epoch 85, Iteration 1000, loss = 0.2255
Checking accuracy on validation set
Got 29043 / 39868 correct (72.85)

Epoch 85, Iteration 1100, loss = 0.1120
Checking accuracy on validation set
Got 28965 / 39868 correct (72.65)

Epoch 85, Iteration 1200, loss = 0.1858
Checking accuracy on validation set
Got 29158 / 39868 correct (73.14)

Epoch 85, Iteration 1300, loss = 0.2080
Checking accuracy on validation set
Got 28778 / 39868 correct (72.18)

Epoch 85, Iteration 1400, loss = 0.2618
Checking accuracy on validation set
Got 28799 / 39868 correct (72.24)

Epoch

Got 28997 / 39868 correct (72.73)

Epoch 88, Iteration 1200, loss = 0.2350
Checking accuracy on validation set
Got 28911 / 39868 correct (72.52)

Epoch 88, Iteration 1300, loss = 0.2847
Checking accuracy on validation set
Got 29043 / 39868 correct (72.85)

Epoch 88, Iteration 1400, loss = 0.3032
Checking accuracy on validation set
Got 28979 / 39868 correct (72.69)

Epoch 88, Iteration 1500, loss = 0.3493
Checking accuracy on validation set
Got 29081 / 39868 correct (72.94)

Epoch 88, Iteration 1600, loss = 0.2400
Checking accuracy on validation set
Got 28923 / 39868 correct (72.55)

Epoch 88, Iteration 1700, loss = 0.1449
Checking accuracy on validation set
Got 28289 / 39868 correct (70.96)

Epoch 88, Iteration 1800, loss = 0.2310
Checking accuracy on validation set
Got 29137 / 39868 correct (73.08)

Epoch 88, Iteration 1900, loss = 0.1443
Checking accuracy on validation set
Got 29094 / 39868 correct (72.98)

Epoch 88, Iteration 2000, loss = 0.2109
Checking accuracy on validation set
G